# Import package

In [32]:
from __future__ import absolute_import
from __future__ import division
import torch.nn.init as init
import torch.nn.functional as F 
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
import pandas as pd
import dgl
from sklearn import preprocessing
import math
from dgl.nn import GATConv
from model import *
from utils import *
from maml import *
import networkx as nx

In [5]:
from torch_geometric.data import DataLoader
import yaml
import time
from datasets import traffic_dataset
device = torch.device('cuda')

# Configure parameters

In [30]:
batch_size = 64
'''----Multi-Type Feature Fusion for User Representation----'''
num_features = 1
num_hidden = 64
s_layers = 2
l_layers = 3
kernel_size_s = 3
kernel_size_l = 7
dropout = 0.2
sigma = 1.0


'''----Multi-granularity Graph Alignment with Feature Fusion----'''
multi_view_len = 2
num_gat_layers = 3
in_dim = 64   # 输入特征维度
hidden_dim = 32
emb_dim = 8
num_heads = 2
emb_epoch = 50


'''----Multi-City Knowledge Transfer Based on Meta-Learning----'''
source_epochs = 50
target_epochs = 120
source_lr = 0.01
model_args['meta_lr'] = 0.001
wd_ft = 0.005
meta_dim = 16
target_days = 3
loss_lambda = 1.5
model = 'GRU'


'''----Load config file----'''
config_filename = 'config.yaml'
test_dataset = 'shanghai'
with open(config_filename) as f:
    config = yaml.safe_load(f)
data_args, task_args, model_args = config['data'], config['task'], config['model']
source_dataset = traffic_dataset(data_args, task_args, "source", test_data=test_dataset)
mean = config['data'][test_dataset]['mean']
std = config['data'][test_dataset]['std']

# Load data

In [ ]:
# input data:
'''
time_series_np         # (num_users=node_nums, sequence_length)
dynamic_attributes     # (num_users=node_nums, num_dynamic_attributes=4, dynamic_attributes_dim=64)
static_attributes      # (num_users=node_nums, static_attributes_dim=10)
source_graphs1 : beijing
source_graphs2 : shanghai
target_graphs  : zhengzhou
'''
# 创建自定义数据集
my_dataset = MyDataset(train_data, dynamic_attributes, static_attributes)
# 使用自定义数据集创建 DataLoader

data_loader = torch.utils.data.DataLoader(my_dataset, batch_size=batch_size, shuffle=True)

# Load Multi-Type Feature Fusion for User Representatio

In [13]:
scaler = preprocessing.StandardScaler()
time_series_np_temp = scaler.fit_transform(time_series_np)
train_data = torch.from_numpy(time_series_np_temp.reshape(time_series_np_temp.shape[0], 1, time_series_np_temp.shape[1]).astype(np.float32))
train_data = torch.Tensor(train_data).to(device)
                              
encoder = DemandEncoder(time_series_np.shape[0], num_features, num_hidden, s_layers, l_layers, kernel_size_s, kernel_size_l, dropout).to(device)
decoder = DemandDecoder(time_series_np.shape[0], num_features, num_hidden, s_layers, l_layers, kernel_size_s, kernel_size_l, dropout).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=0.01)
# Create an instance of the multi-type attributes co-encoder
co_encoder = MultiTypeAttributesCoEncoder(dynamic_attributes_dim=dynamic_attributes_dim, static_attributes_dim=static_attributes_dim, num_dynamic_attributes=num_dynamic_attributes, num_industries=num_industries).to(device)
seq2gauss = seq2gauss_model(num_hidden, num_hidden, device).to(device)

In [27]:
# time-series pre_training
def time_feat_pre_train(encoder, decoder, train_loader, optimizer):
    encoder.train()
    decoder.train()
    train_loss = 0.0
    for i, (data, dynamic_attribute, static_attribute) in enumerate(train_loader):
        z = encoder(data)
        x_hat = decoder(z)
        loss = criterion(x_hat, data)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    return train_loss / len(train_loader)

In [29]:
for epoch in tqdm(range(pre_times_epoch)):
    time_feat_train_loss = time_feat_pre_train(encoder, decoder, data_loader, optimizer)
    print(f"Epoch {epoch+1} - Train Loss: {time_feat_train_loss:.4f}")

# Load Multi-granularity Graph Alignment with Feature Fusion

In [38]:
mvgat = MVGAT(multi_view_len, num_gat_layers, in_dim, hidden_dim, emb_dim, num_heads, True).to(device)
fusion = FusionModule(multi_view_len, emb_dim, 0.8).to(device)
source_agg_graph1 = Agg_Graph(source_graphs[0].num_nodes(), np.load('./data/beijing/matrix.npy').shape[0], emb_dim).to(device)
source_agg_graph2 = Agg_Graph(source_graphs[0].num_nodes(), np.load('./data/shanghai/matrix.npy').shape[0], emb_dim).to(device)
target_agg_graph = Agg_Graph(target_graphs[0].num_nodes(), np.load('./data/shanghai/matrix.npy').shape[0], emb_dim).to(device)
emb_param_list = list(encoder.parameters()) + list(decoder.parameters()) + list(seq2gauss.parameters()) + list(mvgat.parameters()) + list(fusion.parameters()) + list(source_agg_graph1.parameters()) + list(source_agg_graph2.parameters()) + list(target_agg_graph.parameters())
emb_optimizer = optim.Adam(emb_param_list, lr=0.001, weight_decay=0.0001)

In [41]:
decay_lambda = 0.01  # Decay rate hyperparameter
windows_size = 30  # Starting time step for aggregation
source_graph_input1 = [aggregate_dynamic_graphs(source_graphs1, decay_lambda, windows_size).to(device), source_graph_peer.to(device)]
source_graph_input2 = [aggregate_dynamic_graphs(source_graphs2, decay_lambda, windows_size), source_graph_peer.to(device)]
target_graph_input = [target_graphs[0].to(device), target_graph_peer.to(device)]

In [50]:
for i in tqdm(range(emb_epoch)):
    node_embedding_list = []
    for i, (data, dynamic_attribute, static_attribute) in enumerate(data_loader):
        data = data.to(device)
        z = encoder(data)
        # Compute the user attribute encodings
        attribute_encodings = co_encoder(dynamic_attribute, static_attribute)
        node_embedding = seq2gauss(z, attribute_encodings)
        node_embedding_list.append(node_embedding.cpu().detach())
    node_embedding = node_embedding_list[0]
    for i in node_embedding_list[1:]:
        node_embedding = torch.concat((node_embedding, i), dim=0)
    source_graph_feat = node_embedding[0:3881].to(device)
    target_graph_feat = node_embedding[3881:].to(device)
    source_views_city1 = mvgat(source_graph_input1, source_graph_feat)
    source_views_city2 = mvgat(source_graph_input2, source_graph_feat)
    target_views = mvgat(target_graph_input, target_graph_feat)
    source_fused_emb1, source_embs = fusion(source_views_city1)  # 这里得到了每个节点的embedding表示，以及multi-view融合后的embedding表示
    source_fused_emb2, source_embs = fusion(source_views_city2)  # 这里得到了每个节点的embedding表示，以及multi-view融合后的embedding表示
    
    target_fused_emb, target_embs = fusion(target_views)  # 这里得到了每个节点的embedding表示，以及multi-view融合后的embedding表示
    source_zone_embedding1, source_matrix1 = source_agg_graph1(source_fused_emb1)
    source_zone_embedding2, source_matrix2 = source_agg_graph2(source_fused_emb2)
#         print(source_matrix)
    target_zone_embedding1, target_matrix = target_agg_graph(target_fused_emb)
    source_fused_emb =torch.concat((source_fused_emb1, source_fused_emb2), dim=0)
    mmdloss = mmd_loss(source_fused_emb, target_fused_emb,sigma)
    entropy_loss = nn.CrossEntropyLoss()
    entropy_loss = entropy_loss(source_matrix1, source_matrix1) + entropy_loss(source_matrix2, source_matrix2) + entropy_loss(target_matrix, target_matrix)
    loss_global = mmdloss + 0.01 * entropy_loss
    emb_optimizer.zero_grad()
    loss = loss_global
#     print(loss)
    loss.backward()  
    emb_optimizer.step()

# Training and test (Multi-City Knowledge Transfer Based on Meta-Learning)

In [51]:
model_STMAML = STMAML(data_args, task_args, model_args, emb_param_list, model=model).to(device)
source_param_list = emb_param_list + list(model_STMAML.parameters())
source_optimizer = optim.Adam(emb_param_list, lr=source_lr, weight_decay=wd_ft)
    
for epoch in tqdm(range(source_epochs)):
    node_embedding_list = []
    for i, (data, dynamic_attribute, static_attribute) in enumerate(data_loader):
        data = data.to(device)
        z = encoder(data)
        # Compute the user attribute encodings
        attribute_encodings = co_encoder(dynamic_attribute, static_attribute)
        node_embedding = seq2gauss(z, attribute_encodings)
        node_embedding_list.append(node_embedding.cpu().detach())
    node_embedding = node_embedding_list[0]
    for i in node_embedding_list[1:]:
        node_embedding = torch.concat((node_embedding, i), dim=0)
    source_graph_feat = node_embedding[0:3881].to(device)
    target_graph_feat = node_embedding[3881:].to(device)
    source_views_city1 = mvgat(source_graph_input1, source_graph_feat)
    source_views_city2 = mvgat(source_graph_input2, source_graph_feat)
    target_views = mvgat(target_graph_input, target_graph_feat)
    source_fused_emb1, source_embs = fusion(source_views_city1)  
    source_fused_emb2, source_embs = fusion(source_views_city2) 
    
    target_fused_emb, target_embs = fusion(target_views) 
    mmdloss = mmd_loss(source_fused_emb, target_fused_emb,sigma)
    source_zone_embedding1, source_matrix1 = source_agg_graph1(source_fused_emb1)
    source_zone_embedding2, source_matrix2 = source_agg_graph2(source_fused_emb2)
    target_zone_embedding1, target_matrix = target_agg_graph(target_fused_emb)
    source_fused_emb =torch.concat((source_fused_emb1, source_fused_emb2), dim=0)
    entropy_loss = nn.CrossEntropyLoss()
    entropy_loss = entropy_loss(source_matrix1, source_matrix1) + entropy_loss(source_matrix2, source_matrix2) + entropy_loss(target_matrix, target_matrix)
    loss_global = mmdloss + 0.01 * entropy_loss

    spt_task_data, spt_task_A, qry_task_data, qry_task_A = source_dataset.get_maml_task_batch(task_args['task_num'])
    loss = model_STMAML.meta_train_revise(spt_task_data, spt_task_A, qry_task_data, qry_task_A)
    source_optimizer.zero_grad()
    loss.backward()
    source_optimizer.step()
    end_time = time.time()
    print("[Source Train] epoch #{}/{}: loss is {}, training time is {}".format(epoch+1, source_epochs, loss.detach().cpu().numpy(), end_time-start_time))

print("Source dataset meta-train finish.")

metric, output, label =  model_STMAML.finetuning(target_dataloader, test_dataloader, target_epochs, wd_ft, means, stds, path)


In [52]:
# import pickle
# # 读取CSV文件
# # df_post = pd.read_csv('../../zz_relation_value.csv')
# df_post = pd.read_csv('../../sh_relation_value.csv')
# # 将dt列转换为datetime类型，以便进行分组
# df_post['dt'] = pd.to_datetime(df_post['date'])
# # 分组操作，每天构建一张图
# source_graphs = []
# for _, group in tqdm(df_post.groupby('date')):
#     # 获取节点和边的数据
#     src = torch.tensor(group['node1'].to_numpy())
#     dst = torch.tensor(group['node2'].to_numpy())
#     edge_data = torch.tensor(group['rv'].to_numpy())
#     # 创建DGL图
#     bj_g = dgl.graph((src, dst))
#     # 添加边的特征
#     bj_g.edata['rv'] = edge_data
#     # 将图添加到图列表中
#     source_graphs.append(bj_g)
# # f = open('../../bj_time_series_update.pkl','rb')
# f = open('../../sh_time_series_update.pkl','rb')

# bj_time_series = pickle.load(f)
# bj_time_series_list = []
# for i in tqdm(np.array(bj_g.nodes())):
#     time_series = bj_time_series[str(i)]
#     bj_time_series_list.append(time_series)
# bj_time_serie_np = np.array(bj_time_series_list)
# # 读取CSV文件
# # sh_post = pd.read_csv('../../sh_relation_value.csv')
# sh_post = pd.read_csv('../../bj_relation_value.csv')
# # 将dt列转换为datetime类型，以便进行分组
# sh_post['dt'] = pd.to_datetime(sh_post['date'])
# # 分组操作，每天构建一张图
# target_graphs = []
# for _, group in tqdm(sh_post.groupby('date')):
#     # 获取节点和边的数据
#     src = torch.tensor(group['node1'].to_numpy())
#     dst = torch.tensor(group['node2'].to_numpy())
#     edge_data = torch.tensor(group['rv'].to_numpy())
#     # 创建DGL图
#     sh_g = dgl.graph((src, dst))
#     # 添加边的特征
#     sh_g.edata['rv'] = edge_data
#     # 将图添加到图列表中
#     target_graphs.append(sh_g)
# print(sh_g)
# # f = open('../../sh_time_series_update.pkl','rb')
# f = open('../../bj_time_series_update.pkl','rb')

# sh_time_series = pickle.load(f)
# sh_time_series_list = []
# long_term_list = []
# for i in tqdm(np.array(sh_g.nodes())):
#     time_series = sh_time_series[str(i)]
#     sh_time_series_list.append(time_series)
# time_serie_np = np.array(sh_time_series_list)
# # 后3881个数据是target部分的
# time_series_list = []
# for index in bj_time_series_list:
#     time_series_list.append(index)
# for index in sh_time_series_list:
#     time_series_list.append(index)
# time_series_np = np.array(time_series_list)
# f = open('../../industry_l3_id.pkl','rb')
# industry = pickle.load(f)
# for index in range(len(industry)):
#     industry[index] = int(industry[index])
# # source_graphs
# # df_peer = pd.read_csv('../../0420_bj_relation_peer_update_l3_area.csv')
# df_peer = pd.read_csv('../../0422_sh_relation_peer_update_L3_area.csv')
# # 获取节点和边的数据
# src = torch.tensor(df_peer['node1'].to_numpy())
# dst = torch.tensor(df_peer['node2'].to_numpy())
# # edge_data = torch.tensor(df_peer['rv'].to_numpy())
# # 创建DGL图
# source_graph_peer = dgl.graph((src, dst))
# # 添加边的特征
# # source_graph_peer.edata['rv'] = edge_data
# # target_graphs
# # df_peer = pd.read_csv('../../0422_sh_relation_peer_update_L3_area.csv')
# df_peer = pd.read_csv('../../0420_bj_relation_peer_update_l3_area.csv')
# # 获取节点和边的数据
# src = torch.tensor(df_peer['node1'].to_numpy())
# dst = torch.tensor(df_peer['node2'].to_numpy())
# # edge_data = torch.tensor(df_peer['rv'].to_numpy())
# # 创建DGL图
# target_graph_peer = dgl.graph((src, dst))
# # 添加边的特征
# # target_graph_peer.edata['rv'] = edge_data
# # 读取CSV文件
# # df_post = pd.read_csv('../../bj_relation_value.csv')
# df_post = pd.read_csv('../../sh_relation_value.csv')
# # 将dt列转换为datetime类型，以便进行分组
# df_post['dt'] = pd.to_datetime(df_post['date'])
# # 分组操作，每天构建一张图
# source_graphs = []
# for _, group in tqdm(df_post.groupby('date')):
#     # 获取节点和边的数据
#     src = torch.tensor(group['node1'].to_numpy())
#     dst = torch.tensor(group['node2'].to_numpy())
# #     edge_data = torch.tensor(group['rv'].to_numpy())
#     # 创建DGL图
#     g = dgl.graph((src, dst))
#     # 添加边的特征
# #     g.edata['rv'] = edge_data
# #     g.ndata['feat'] = node_embedding.cpu()[0:12044]
# #     g = dgl.add_self_loop(g)
#     # 将图添加到图列表中
#     source_graphs.append(g)
# # 读取CSV文件
# # sh_post = pd.read_csv('../../sh_relation_value.csv')
# sh_post = pd.read_csv('../../bj_relation_value.csv')
# # 将dt列转换为datetime类型，以便进行分组
# sh_post['dt'] = pd.to_datetime(sh_post['date'])
# # 分组操作，每天构建一张图
# target_graphs = []
# for _, group in tqdm(sh_post.groupby('date')):
#     # 获取节点和边的数据
#     src = torch.tensor(group['node1'].to_numpy())
#     dst = torch.tensor(group['node2'].to_numpy())
# #     edge_data = torch.tensor(group['rv'].to_numpy())
#     # 创建DGL图
#     sh_g = dgl.graph((src, dst))
#     # 添加边的特征
# #     sh_g.edata['rv'] = edge_data
# #     sh_g.ndata['feat'] = node_embedding.cpu()[12044:]
# #     sh_g = dgl.add_self_loop(sh_g)
#     # 将图添加到图列表中
#     target_graphs.append(sh_g)
# print(sh_g)